## **Exercise SEP Developer Test**

This test is designed to assess your ability to design and develop a project from a limited set of goals, provided below. We will evaluate your project based on its fulfillment of the outlined objectives, code efficiency, readability, and adherence to best practices. Additionally, we will evaluate the structure and content of your GitHub repository according to standard practices.

1. Please perform all coding within the Google Colab environment: http://colab.research.google.com
    * Please store your project in a GitHub repository, so we can fork and run it in Colab.
        * Additionally, please create a standard GitHub repository with the expected files and directory structure.    
    * Please note that Colab doesn't automatically include additional files. To avoid potential issues, we recommend only including files that the evaluators already have access to.    
    * The evaluators will use the files SEP01.mseed, SEP02.mseed, and SEP03.mseed, which can be found in this GitHub project.
2. Import the mseed files
3. Create a database with proper normal form and constraints
4. Import the data into the newly created database
5. Create a visualization for the data. The visualization can be static but must include:    
    * A title
    * Text section displaying metadata about the miniseed data
    * Helicorder-style charts of the miniseed data (These may be built from the database data or the mseed files)
    * A map with icons indicating station locations based on the miniseed data (station lat and lon can be found on iris' API)
    * Add to the map additional stations found on IRIS' data API. Stations of interest include HOA and SUG
        * Data API information can be found at http://service.iris.edu/fdsnws/station/1/



## **Environment Setup**


### **Overview**

This section installs all necessary packages used to run the code. Please run the **Installation Code** section to install all necessary packages for the current session.

Packages used in this project include:
- **ObsPy** - used for seismic data processing and analysis.
- **Folium** - used for creating interactive maps of seismic station locations.
- **SQLAlchemy** (installed via ObsPy) - supports database operations for managing seismic station and waveform data.

### **Installation Code**

In [85]:
# Install necessary quietly
!pip install -q obspy folium > /dev/null 2>&1
# Confirm successful installation
print("All Packages Successfully Installed.")

All Packages Successfully Installed.


## **Database Creation**


### **Overview**

This section initializes and tests the database creation functions. Be sure to run the **Database Creation Code** section to set up the database functions for the current session.

The database is built using SQLAlchemy, which integrates seamlessly with Python objects. It is configured as a file-based structure named seismic_data.db. Unit tests are included to verify the database structure and ensure querying functionality operates as expected.


#### **Database Structure**

1. **Station Table**:
- Stores metadata about seismic stations.
- Columns:
 - `station_id`: Unique identifier for each station (Primary Key).
 - `station`: Name of the station.
 - `network`: Network code.
 - `channel`: Channel code (e.g., EHZ).
 - `latitude`: Latitude of the station.
 - `longitude`: Longitude of the station.

2. **WaveformTrace Table**:
- Stores individual seismic waveform traces.
- Columns:
 - `trace_id`: Unique identifier for each trace (Primary Key).
 - `station_id`: Links to the corresponding station (Foreign Key).
 - `start_time`: Start time of the trace.
 - `end_time`: End time of the trace.
 - `trace_data`: Serialized binary data of the waveform.
 - `sampling_rate`: Sampling rate of the waveform in Hz.

#### **Database Functions**

1. **setup_database**()

- Initializes a SQLite database with tables for Station and WaveformTrace.
Configures the database as a file-based structure (seismic_data.db) and returns an engine and session for interaction.

2. **initialize_database**(stream, session)

- Populates the database using an ObsPy Stream object.
Adds station metadata if it is not already in the database and stores individual seismic waveform traces, including metadata like sampling rate and timestamps.

3. **get_coordinates**(station_name)

- Retrieves latitude and longitude coordinates for a station using the IRIS data API.
If unavailable via IRIS, prompts the user to manually input the missing coordinates.

#### **Database Unit Tests Functions**

1. **setup_database**()

- Sets up the database for unit testing by creating tables for Station and WaveformTrace.
Uses the file-based structure (seismic_data.db) for accurate testing.

2. **test_full_trace_count**()

- Confirms all traces from the ObsPy Stream are stored in the database.
Validates that the total number of traces matches the expected count.

3. **test_query_by_time_range**()

- Tests the ability to query traces by a specific time range.
Verifies that the number of returned traces aligns with the expected results for the queried window.

4. **test_query_by_station**()

- Ensures that the database correctly filters traces by station name.
Confirms all retrieved traces belong to the specified station.

5. **test_query_by_sampling_rate**()

- Checks the filtering of traces by sampling rate.
Validates that all retrieved traces match the given sampling rate.

6. **test_query_by_channel**()

- Verifies the database can filter traces by channel.
Ensures all returned traces belong to the queried channel.

7. **test_query_by_location**()

- Confirms the ability to filter traces by geographic coordinates (latitude and longitude).
Validates that the returned traces correspond to the specified location.

8. **test_validate_trace_metadata**()

- Verifies the integrity of trace metadata stored in the database.
Checks values like start time, end time, sampling rate, and data length against known expected values to ensure accuracy.

### **Database Creation Code**

In [91]:
import os
from obspy import Trace, Stream, UTCDateTime, read
from obspy.clients.fdsn import Client
from sqlalchemy import create_engine, Column, Integer, String, Float, LargeBinary, ForeignKey
from sqlalchemy.util import deprecations
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pickle

#This database uses the version of SQLAlchemy compatible with Obspy as opposed to the newest version
deprecations.SILENCE_UBER_WARNING = True
#Environment Setup
Base = declarative_base()
client = Client("IRIS")

class Station(Base):
    __tablename__ = 'stations'
    station_id = Column(Integer, primary_key=True, autoincrement=True)
    station = Column(String, nullable=False)
    network = Column(String, nullable=False)
    channel = Column(String, nullable=False)
    latitude = Column(Float, nullable=True)
    longitude = Column(Float, nullable=True)

class WaveformTrace(Base):
    __tablename__ = 'waveform_traces'
    trace_id = Column(Integer, primary_key=True, autoincrement=True)
    station_id = Column(Integer, ForeignKey('stations.station_id'), nullable=False)
    start_time = Column(String, nullable=False)
    end_time = Column(String, nullable=False)
    trace_data = Column(LargeBinary, nullable=False)
    sampling_rate = Column(Float, nullable=False)

def setup_database():
    #Creates database named "sqlite:///seismic_data.db"
    #create_engine("sqlite:///:memory:") could be used for faster processing but less scalability
    #Remove seismic_data.db if it already exists

    if os.path.exists("seismic_data.db"):
      os.remove("seismic_data.db")
    engine = create_engine("sqlite:///seismic_data.db")
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    return engine, session

def get_coordinates(station_name):
    #Pulls coordinates from IRIS API using station name or user input if unavailable

    latitude = None
    longitude = None
    try:
        inventory = client.get_stations(station=station_name, level="station")
        station_info = inventory[0][0]
        latitude = station_info.latitude
        longitude = station_info.longitude
    except:
        print(f"Unable to retrieve coordinates automatically:")
        while latitude is None:
            try:
                latitude = float(input("Enter latitude: "))
            except ValueError:
              print("Invalid input. Please enter a numeric value for latitude.")
            while longitude is None:
              try:
                longitude = float(input("Enter longitude: "))
              except ValueError:
                print("Invalid input. Please enter a numeric value for longitude.")
    return latitude, longitude

def initialize_database(stream, session):
    #Initialize database with stream and session

    for trace in stream:
        station_entry = session.query(Station).filter_by(
            station=trace.stats.station, network=trace.stats.network
        ).first()
        #.mseed data may not have location
        if not station_entry:
            latitude = None
            longitude = None

            if hasattr(trace.stats, 'latitude') and hasattr(trace.stats, 'longitude'):
                latitude = trace.stats.latitude
                longitude = trace.stats.longitude
            else:
                #Search IRIS for station location
                latitude, longitude = get_coordinates(trace.stats.station)

            #Add station to database
            station_entry = Station(
                station=trace.stats.station,
                network=trace.stats.network,
                channel=trace.stats.channel,
                latitude=latitude,
                longitude=longitude
            )
            session.add(station_entry)
            session.commit()

        #Add each trace to the database
        new_trace = WaveformTrace(
            station_id=station_entry.station_id,
            start_time=str(trace.stats.starttime),
            end_time=str(trace.stats.endtime),
            trace_data=pickle.dumps(trace.data),
            sampling_rate=trace.stats.sampling_rate
        )
        session.add(new_trace)

    session.commit()
    print("Database initialized successfully.")


print("Database creation functions have been populated. Please proceed to the next step.")

Database creation functions have been populated. Please proceed to the next step.


### **Unit Test Code**

In [100]:
import unittest
from obspy import Stream, read
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pickle
from datetime import datetime
import os

def setup_database():
    #Database setup from the previous code for unit tests

    #Remove db if already exists for retesting
    if os.path.exists("seismic_data.db"):
      os.remove("seismic_data.db")
    engine = create_engine("sqlite:///seismic_data.db")
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    return engine, session

class VerboseTestResult(unittest.TextTestResult):
    #Add custom success and failure messages

    def addSuccess(self, test):
        super().addSuccess(test)
        self.stream.writeln(f"✔️ SUCCESS: {test.shortDescription() or str(test)}")

    def addFailure(self, test, err):
        super().addFailure(test, err)
        self.stream.writeln(f"❌ FAILURE: {test.shortDescription() or str(test)}")

class VerboseTestRunner(unittest.TextTestRunner):
    def _makeResult(self):
        return VerboseTestResult(self.stream, self.descriptions, self.verbosity)

class TestDatabase(unittest.TestCase):
    def setUp(self):
        #Setup database and session for each test
        self.engine, self.session = setup_database()

        #Load traces directly from .mseed files
        self.stream = Stream()
        for file in ["SEP01.mseed", "SEP02.mseed", "SEP03.mseed"]:
            try:
                self.stream += read(file)
            except Exception as e:
                print(f"Error reading file {file}: {e}")

        #Initialize database with these traces
        initialize_database(self.stream, self.session)

    def test_full_trace_count(self):
        "Query by Full Trace Count"

        #Query database for all traces
        traces = self.session.query(WaveformTrace).all()
        #Verify all traces are in the database
        self.assertEqual(len(traces), 32, "All traces should be in the database")

    def test_query_by_time_range(self):
        "Query by Time Range"

        #Query based on a specific time range
        start_time = "2023-11-11T00:00:00.000001Z"
        end_time = "2023-11-11T02:31:38.090001Z"

        traces = self.session.query(WaveformTrace).filter(
            WaveformTrace.start_time >= start_time,
            WaveformTrace.end_time <= end_time
        ).all()

        #Expected number of traces in this range
        expected_count = 7
        self.assertEqual(len(traces), expected_count, f"Expected {expected_count} traces in the given time range.")

    def test_query_by_station(self):
        "Query by Station"

        #Query based on station name
        station_name = "SEP"
        traces = self.session.query(WaveformTrace).join(Station).filter(Station.station == station_name).all()

        #Verify all traces belong to the station "SEP"
        self.assertEqual(len(traces), 32, "All 32 traces should belong to station 'SEP'.")

    def test_query_by_sampling_rate(self):
        "Query by Sampling Rate"

        #Query based on a specific sampling rate
        sampling_rate = 100.0
        traces = self.session.query(WaveformTrace).filter(WaveformTrace.sampling_rate == sampling_rate).all()

        #Verify all traces have a sampling rate of 100.0 Hz
        self.assertEqual(len(traces), 32, "All 32 traces should have a sampling rate of 100.0 Hz.")

    def test_query_by_channel(self):
        "Query by Channel"

        #Query based on a specific channel
        channel = "EHZ"
        traces = self.session.query(WaveformTrace).join(Station).filter(Station.channel == channel).all()

        #Verify all traces belong to the channel "EHZ"
        self.assertEqual(len(traces), 32, "All 32 traces should belong to channel 'EHZ'.")

    def test_query_by_location(self):
        "Query by Location"

        #Query based on station latitude and longitude
        latitude = 46.19978
        longitude = -122.190857
        traces = self.session.query(WaveformTrace).join(Station).filter(
            Station.latitude == latitude,
            Station.longitude == longitude
        ).all()

        #Verify all traces have the correct coordinates
        self.assertEqual(len(traces), 32, "All traces should belong to the specified location.")

    def test_validate_trace_metadata(self):
        "Validation of Trace Metadata"

        #Query all traces
        traces = self.session.query(WaveformTrace).all()

        #Verify metadata for the first 7 traces
        expected_values = [
            ("2023-11-11T00:00:00.000001Z", "2023-11-11T00:36:58.090001Z", 100.0, 221810),
            ("2023-11-11T00:37:45.100001Z", "2023-11-11T01:41:06.090001Z", 100.0, 380100),
            ("2023-11-11T01:43:07.100001Z", "2023-11-11T02:01:05.090001Z", 100.0, 107800),
            ("2023-11-11T02:01:47.100001Z", "2023-11-11T02:02:10.090001Z", 100.0, 2300),
            ("2023-11-11T02:02:24.100001Z", "2023-11-11T02:17:01.090001Z", 100.0, 87700),
            ("2023-11-11T02:17:49.100001Z", "2023-11-11T02:31:16.090001Z", 100.0, 80700),
            ("2023-11-11T02:31:37.100001Z", "2023-11-11T02:31:38.090001Z", 100.0, 100)
        ]

        for trace, expected in zip(traces[:7], expected_values):
            self.assertEqual(trace.start_time, expected[0], "Start time does not match")
            self.assertEqual(trace.end_time, expected[1], "End time does not match")
            self.assertEqual(trace.sampling_rate, expected[2], "Sampling rate does not match")
            self.assertEqual(len(pickle.loads(trace.trace_data)), expected[3], "Data length does not match")

if __name__ == "__main__":
    unittest.main(testRunner=VerboseTestRunner, argv=[''], exit=False)

.

Database initialized successfully.


✔️ SUCCESS: Query by Full Trace Count
.

Database initialized successfully.


✔️ SUCCESS: Query by Channel
.

...

Database initialized successfully.


✔️ SUCCESS: Query by Station
.

Database initialized successfully.


✔️ SUCCESS: Query by Time Range
.✔️ SUCCESS: Validation of Trace Metadata

----------------------------------------------------------------------
Ran 7 tests in 9.267s

OK


Database initialized successfully.


## **Visualization of Seismic Data**

### **Overview**



This section initializes visual creation functions. Be sure to run the **Visualization Code** section to set up the visualization functions for the current session. Using a combination of packages, including matplotlib for plotting helicorder charts and folium for creating interactive maps, this section ensures that seismic data is presented in a visually engaging and easy-to-understand format, with options for downloading the results.


#### **Visualization Functions**

1. **display_all**(station, stream)

- Creates an interactive visualization for a seismic station's data, displaying metadata, a map, and helicorder plots for each unique day in separate tabs. Includes functionality to download each day's visualization as an HTML file.

2. **create_metadata_chart**(station, daily_stream)

- Generates an HTML widget displaying metadata for a given station and daily stream, including network, station, channel, coordinates, sampling rate, and trace count.

3. **plot_helicorder**(station, stream, day)

- Produces a helicorder plot for a specified day using ObsPy, formatting it with a title, axis labels, and gridlines. The plot is embedded as a base64-encoded PNG within an HTML widget.

4. **create_map**(station)

- Creates a folium map centered on the station's location, adding predefined nearby stations and a legend for marker types. Outputs the map as an HTML widget.

5. **add_stations_to_map**(map_object, main_station_coords, main_station, distance=None)

- Adds markers for predefined nearby stations (e.g., HOA and SUG) to the given folium map, including metadata such as coordinates and distance from the main station.

6. **save_html_to_file**(content, filename="visualization_output.html")

- Saves HTML content to a specified file, enabling the visualization to be stored and shared.

7. **create_download_button**(full_html, filename)

- Generates a button widget that saves the provided HTML content to a file when clicked, allowing users to download visualizations directly.

8. **visualize_from_query**(station_name, session, start_datetime=None, end_datetime=None)

- Queries a database for seismic traces matching the given station and time range, then visualizes the results using the display_all function.

### **Visualization Code**

In [94]:
from obspy import Trace, Stream, UTCDateTime, read
from obspy.clients.fdsn import Client
from sqlalchemy import create_engine, Column, Integer, String, Float, LargeBinary, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from geopy.distance import geodesic
import pickle
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import HTML, VBox, Tab, HBox, Layout, Button, Label
import folium
import io
import base64
import matplotlib.pyplot as plt
from datetime import datetime

def visualize_from_query(station_name, session, start_datetime=None, end_datetime=None):
    #Query database for data and generate seismic data visualization

    station = session.query(Station).filter_by(station=station_name).first()
    if not station:
        print("Station not found in database.")
        return

    traces = session.query(WaveformTrace).filter_by(station_id=station.station_id).all()
    filtered_stream = Stream()

    for trace in traces:
        if start_datetime or end_datetime:
            if start_datetime and UTCDateTime(trace.start_time) <= start_datetime:
                continue
            if end_datetime and UTCDateTime(trace.end_time) >= end_datetime:
                continue
        data = pickle.loads(trace.trace_data)
        new_trace = Trace(data=data, header={
            'starttime': UTCDateTime(trace.start_time),
            'endtime': UTCDateTime(trace.end_time),
            'station': station.station,
            'network': station.network,
            'sampling_rate': trace.sampling_rate
        })
        filtered_stream.append(new_trace)

    if not filtered_stream:
        print("No matching traces found.")
    else:
        display_all(station, filtered_stream)

def display_all(station, stream):
    #Creates a html to display with tabs for each day including:
    #Metadata, Map, and Helicorder

    #Clears out output for better visualization
    clear_output(wait=True)
    #Generate tabs for each unique day
    unique_days = {}
    for trace in stream:
        day = trace.stats.starttime.date
        if day not in unique_days:
            unique_days[day] = Stream()
        unique_days[day].append(trace)

    tabs = Tab()
    tab_children = []
    #Generate data for each day
    for idx, (day, daily_stream) in enumerate(unique_days.items()):
        # Create Metadata
        metadata_widget = create_metadata_chart(station, daily_stream)
        metadata_html = metadata_widget.value

        # Create Map
        map_widget = create_map(station)
        map_html = map_widget.value

        # Generate Helicorder Plot
        helicorder_widget = plot_helicorder(station, daily_stream, day)
        helicorder_html = helicorder_widget.value

        #Combine all content into a single HTML and format appropriately
        day_title = datetime.strptime(str(day), "%Y-%m-%d").strftime("%b %d %Y")
        day_filename = datetime.strptime(str(day), "%Y-%m-%d").strftime("%b_%d_%Y")
        full_html = f"""
        <html>
        <head>
            <title>Seismic Data Visualization for {day_title}</title>
            <style>
                body {{ font-family: Arial, sans-serif; }}
                .title {{ text-align: center; font-weight: bold; font-size: 24px; margin-top: 20px; }}
                .metadata {{ float: left; width: 25%; padding: 10px; }}
                .map {{ float: right; width: 70%;  padding: 10px; }}
                .helicorder {{ clear: both; margin-top: 20px; text-align: center; }}
            </style>
        </head>
        <body>
            <div class="title">Seismic Data Visualization for {day_title}</div>
            <div class="metadata">{metadata_html}</div>
            <div class="map">{map_html}</div>
            <div class="helicorder">{helicorder_html}</div>
        </body>
        </html>
        """

        #Create title and download button
        download_button = create_download_button(full_html, f"Seismic_Data_Visualization_{day_filename}.html")
        title_with_button = HBox([
            HTML(value=f"<h3 style='text-align: center; font-weight: bold; font-size: 20px; margin: 0;'>Seismic Data Visualization for {day_title}</h3>"),
            download_button
        ], layout=Layout(justify_content="space-between", align_items="center", width="100%"))

        #Arrange layout so Metadata and Map are together with Helicorder below
        metadata_widget.layout = Layout(
            width="25%",  # 25% width for metadata
            height="100%",
            overflow="auto"
        )

        map_widget.layout = Layout(
            width="75%",  # 75% width for the map
            height="100%",
        )

        tab_content = VBox([
            title_with_button,
            HBox([metadata_widget, map_widget]),
            helicorder_widget
        ])
        tab_children.append(tab_content)

    tabs.children = tab_children
    for i, day in enumerate(unique_days.keys()):
        day_title = datetime.strptime(str(day), "%Y-%m-%d").strftime("%b %d %Y")
        tabs.set_title(i, day_title)

    display(tabs)

def create_metadata_chart(station, daily_stream):
    #Extract Metadata
    start_time = min(trace.stats.starttime for trace in daily_stream)
    end_time = max(trace.stats.endtime for trace in daily_stream)

    #Create Metadata html with proper formatting
    metadata_html = f'''
    <div style="padding:0px; background-color:#f9f9f9; box-shadow:0 2px 5px rgba(0,0,0,0.1);border: 3px solid black; overflow: auto;">
        <table style="width:100%; border-spacing:0px; font-size:12px; line-height:2.08;">
            <tr><th colspan="2" style="font-size:14px; margin-bottom:5px;"><h3 style="margin:0;">Station Metadata</h3></th></tr>
            <tr><td><b>Network:</b></td><td>{station.network}</td></tr>
            <tr><td><b>Station:</b></td><td>{station.station}</td></tr>
            <tr><td><b>Channel:</b></td><td>{station.channel}</td></tr>
            <tr><td><b>Coordinates:</b></td><td>({station.latitude}, {station.longitude})</td></tr>
            <tr><td><b>Sampling Rate:</b></td><td>{daily_stream[0].stats.sampling_rate} Hz</td></tr>
            <tr><td><b>Start Time:</b></td><td>{start_time}</td></tr>
            <tr><td><b>End Time:</b></td><td>{end_time}</td></tr>
            <tr><td><b>Trace Count:</b></td><td>{len(daily_stream)}</td></tr>
        </table>
    </div>
    '''
    return HTML(value=metadata_html)

def plot_helicorder(station, stream, day):
    #Generate the helicorder plot and format it using matplotlib

    #Interpolate data linearly over gaps in traces and create dayplot
    filled_stream = stream.merge(method=1, fill_value='interpolate')
    fig = filled_stream.plot(type='dayplot',
              interval=60,
              method='',
              one_tick_per_line=True,
              time_axis_bottom=False,
              linewidth = .5,
              number_of_ticks=11,
              size=(900, 1500),
              title="",
              handle=True
    )

    day_title = datetime.strptime(str(day), "%Y-%m-%d").strftime("%b %d %Y")

    #Add Plot Titles and Labels
    ax = fig.gca()
    ax.set_title("Helicorder Plot for " + station.station +'.' + station.network +
                 '.' + station.channel + " on " + day_title, fontsize=18)
    ax.set_xlabel("Time (Minutes)", fontweight="bold", fontsize=16)
    ax.set_ylabel("UTC Time", fontsize=16)
    ax.grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.7)
    ax.tick_params(axis="both", which="major", labelsize=10)
    ax.tick_params(axis="both", which="minor", labelsize=8)
    plt.tight_layout()

    #Add to buffer and encode
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    base64_image = base64.b64encode(buffer.read()).decode('utf-8')
    buffer.close()
    plt.close(fig)


    # Create an HTML element for the helicorder plot
    helicorder_html = f'''
    <div style="border: 3px solid black; margin-top: 0px;">
        <img src="data:image/png;base64,{base64_image}" style="width: 100%; height: auto;">
    </div>
    '''
    return HTML(helicorder_html)

def create_map(station):
    #Create the map using folium

    map_object = folium.Map(location=(station.latitude, station.longitude), zoom_start=10, zoom_control=False, height="100%", width="100%")
    folium.Marker(
        location=(station.latitude, station.longitude),
        popup=(
                    f"<b>Station:<br></b> {station.station}<br>"
                    f"<b>Network:<br></b> {station.network}<br>"
                    f"<b>Coordinates:</b> ({round(station.latitude,3)}, {round(station.longitude,3)})<br>"
                ),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map_object)

    #Add additional stations to the map
    add_stations_to_map(map_object, (station.latitude, station.longitude), station.station)

    #Add legend
    legend_html = '''
      <div style="position: fixed;
                  bottom: 0px; left: 0px; width: 150px; height: 90px;
                  background-color: white; z-index:9999;
                  border-top: 3px solid black; border-right: 3px solid black; padding:10px; font-size:12px;
                  box-shadow:3px 3px 3px rgba(0,0,0,0.3);">
        <h4 style="margin:0;">Legend</h4>
        <p style="margin:0;"><i style="color:red;" class="fa fa-map-marker"></i> Main Station</p>
        <p style="margin:0;"><i style="color:blue;" class="fa fa-map-marker"></i> Nearby Station</p>
      </div>
      '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

    #Add and format title
    title_html = '''
    <div style="position: fixed; text-align: center; font-weight: bold;
            top: 0px; left: 50%; transform: translate(-50%, 0);
            z-index:9999; font-size:16px;
            width: 100%; background:white; padding:5px 10px; border-bottom: 3px solid black;">
    <b>Station Map: SEP</b>
    </div>
    '''
    map_object.get_root().html.add_child(folium.Element(title_html))


    # Convert the map into an HTML widget
    map_html = f'<div style="border: 3px solid black;">{map_object._repr_html_()}</div>'
    return HTML(value=map_html)

def add_stations_to_map(map_object, main_station_coords, main_station):
    #Add stations to the map

    #Add IRIS client to pull data
    client = Client("IRIS")

    #Predefined stations (HOA, SUG)
    predefined_stations = [
        {"station": "HOA", "network": "CC"},
        {"station": "SUG", "network": "CC"}
    ]

    #Pull data from IRIS
    for station in predefined_stations:
        try:
            inventory = client.get_stations(network=station["network"], station=station["station"])
            station_info = inventory[0][0]
            lat, lon = station_info.latitude, station_info.longitude
            distance = geodesic(main_station_coords, (lat, lon)).miles
            folium.Marker(
                location=(lat, lon),
                popup=(
                    f"<b>Station:</b> <br>{station['station']}<br>"
                    f"<b>Network:</b> <br>{station['network']}<br>"
                    f"<b>Coordinates:</b> ({round(lat,3)}, {round(lon,3)})<br>"
                    f"<b>Distance to {main_station}</b> <br> {distance:.2f} miles"
                ),
                icon=folium.Icon(color="blue", icon="info-sign")
            ).add_to(map_object)
        except Exception as e:
            print(f"Could not fetch data for station {station['station']}: {e}")

def save_html_to_file(content, filename):
    #Save html to file for downloading visualization
    with open(filename, "w") as f:
        f.write(content)

def create_download_button(full_html, filename):
    #Save html to file for downloading visualization

    #Create event for downloading files instead of autodownloading them
    def download_action(_):
        save_html_to_file(full_html, filename)
        print(f"Saved {filename} for download.")

    button = Button(
        description="Download",
        button_style="primary",
        tooltip=f"Download {filename}",
        layout=Layout(width="100px", height="30px")
    )
    button.on_click(download_action)
    return button

print("Visualization functions have been populated. Please proceed to the next step.")


Visualization functions have been populated. Please proceed to the next step.
